<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project_google_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kütüphaneler

In [ ]:
!pip install -q torch accelerate transformers datasets huggingface-hub

## Modellere göre sınıflandırma (Çözebilir mi?)

Modeller:

https://huggingface.co/google/gemma-2-9b-it


### Modelin yüklenmesi

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

token = userdata.get('HF_TOKEN')
login(token=token)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
google_gemma = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

# pipeline'ları oluşturma
#google_gemma = pipeline(
#    "text-generation",
#    model="google/gemma-2-9b-it",
#    model_kwargs={"torch_dtype": torch.bfloat16},
#    device="cuda",  # replace with "mps" to run on a Mac device
#)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Değerlendirme fonksiyonu

In [ ]:
import re

def extract_number(text):
    # Metindeki sayıları bul, sonuncusunu float’a çevir. Son sayı cevap.
    nums = re.findall(r"[-+]?\d+\.?\d*", text.replace(",","."))
    return float(nums[-1]) if nums else None

def predict_gg_gemma(question):
  prompt = (
    f"Soru: {question}\n"
    "Soruya tek bir yanıt ver, adım adım çözme, sadece sonuç olsun.",
    "Yanıtlarında ‘24 \times \frac’ gibi notasyonlar kullanma.",
    "Bunu da ‘Sonuç: <değer>’ formatında göster."
  )

  messages = [
    {"role": "user", "content": prompt},
  ]

  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,  # Added tokenize=True
    return_tensors="pt",
  ).to("cuda")

  outputs = google_gemma.generate(
    input_ids,
    max_new_tokens=256,
    do_sample=False,
  )

  # Assuming outputs is a tensor now
  response = outputs[0][input_ids.shape[-1]:]
  assistant_response = tokenizer.decode(response, skip_special_tokens=True)
  return assistant_response


def label_prediction(question, true_answer,):
  gg_gemma_result = predict_gg_gemma(question)
  gg_gemma_prediction = extract_number(gg_gemma_result)

  real = extract_number(true_answer)
  # Tahmin gerçek değerle eşleşiyorsa 1, yoksa 0
  is_gg_gemma_correct = 1 if (gg_gemma_prediction is not None and real is not None and abs(gg_gemma_prediction-real)<1e-3) else 0
  return gg_gemma_result, is_gg_gemma_correct

### Örnek Soru-Cevap Karşılaştırması

In [ ]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "info"

df = pd.read_csv("gsm8k_tr_classified.csv")

results = []

# Sadece ilk 5 kayıt için döngü
for i in range(5):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  gg_gemma_result, is_gg_gemma_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunlar ekleyelim
  df.loc[i, "gg_gemma_result"] = gg_gemma_result
  df.loc[i, "is_gg_gemma_correct"] = is_gg_gemma_correct

# Sonuçları DataFrame olarak görüntüle (İlk 5 satır için)
display(df.head())

/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


,question,answer,question_type,solution_method,gg_gemma_result,is_gg_gemma_correct
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama,Sonuç: 270 \n,0.0
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma,Sonuç: 21 \n,1.0
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran,Sonuç: 24 \n,1.0
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan çarpma,Sonuç: 90 \n,0.0
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran,Sonuç: 18 \n,1.0


In [ ]:
from tqdm.notebook import tqdm
import os

os.environ["TRANSFORMERS_VERBOSITY"] = "info"


# Tüm DataFrame üzerinde döngü
for i in tqdm(range(len(df)), desc="Processing rows"):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  gg_gemma_result, is_gg_gemma_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunları ekleyelim
  df.loc[i, "gg_gemma_result"] = gg_gemma_result
  df.loc[i, "is_gg_gemma_correct"] = is_gg_gemma_correct

  # Her 100 iterasyonda bir veya döngünün sonunda kaydet
  if (i + 1) % 100 == 0 or (i + 1) == len(df):
    df.to_csv("gsm8k_tr_classified_with_gemma_results.csv", index=False)
    print(f"{i+1} kayıt işlendi ve kaydedildi.")


print("Sonuçlar 'gsm8k_tr_classified_with_gemma_results.csv' dosyasına kaydedildi.")

Processing rows:   0%|          | 0/8792 [00:00<?, ?it/s]

100 kayıt işlendi ve kaydedildi.
200 kayıt işlendi ve kaydedildi.
300 kayıt işlendi ve kaydedildi.
400 kayıt işlendi ve kaydedildi.
500 kayıt işlendi ve kaydedildi.
600 kayıt işlendi ve kaydedildi.
700 kayıt işlendi ve kaydedildi.
800 kayıt işlendi ve kaydedildi.
900 kayıt işlendi ve kaydedildi.
1000 kayıt işlendi ve kaydedildi.
1100 kayıt işlendi ve kaydedildi.
1200 kayıt işlendi ve kaydedildi.
1300 kayıt işlendi ve kaydedildi.
1400 kayıt işlendi ve kaydedildi.
1500 kayıt işlendi ve kaydedildi.
1600 kayıt işlendi ve kaydedildi.
1700 kayıt işlendi ve kaydedildi.
1800 kayıt işlendi ve kaydedildi.
1900 kayıt işlendi ve kaydedildi.
2000 kayıt işlendi ve kaydedildi.
2100 kayıt işlendi ve kaydedildi.
2200 kayıt işlendi ve kaydedildi.
2300 kayıt işlendi ve kaydedildi.
2400 kayıt işlendi ve kaydedildi.
2500 kayıt işlendi ve kaydedildi.
2600 kayıt işlendi ve kaydedildi.
2700 kayıt işlendi ve kaydedildi.
2800 kayıt işlendi ve kaydedildi.
2900 kayıt işlendi ve kaydedildi.
3000 kayıt işlendi ve k

In [ ]:
# Calculate the accuracy of the Gemma model
accuracy = df['is_gg_gemma_correct'].mean() * 100
print(f"Gemma Model Doğruluğu: {accuracy:.2f}%")

# Display some examples where the model was incorrect
incorrect_predictions = df[df['is_gg_gemma_correct'] == 0]
print("\nYanlış Tahmin Edilen Örnekler:")
display(incorrect_predictions[['question', 'answer', 'gg_gemma_result']].head())

# You could also analyze the distribution of correct vs incorrect predictions
correct_counts = df['is_gg_gemma_correct'].value_counts()
print("\nDoğru/Yanlış Tahmin Dağılımı:")
correct_counts


Gemma Model Doğruluğu: 17.72%

Yanlış Tahmin Edilen Örnekler:


,question,answer,gg_gemma_result
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,Sonuç: 270 \n
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,Sonuç: 90 \n
5,Sinema biletlerinin tanesi Pazartesi günü 5 do...,"Glenn Çarşamba günü 10 dolar, Cumartesi günü i...",Sonuç: 40 \n
6,Conner'ın çölde sürdüğü bir kumul arabası var....,Conner'ın düz kumda hızı 60 mil/saat.\nYokuş a...,Sonuç: 48 \n
8,James arabasını değiştirmeye karar verir. 20....,"James arabasını 20.000 $'ın %80'ine, yani 16.0...",Sonuç: 6000 \n



Doğru/Yanlış Tahmin Dağılımı:


,count
is_gg_gemma_correct,
0.0,7234
1.0,1558
